In [117]:
import pandas as pd

In [118]:
xl = pd.ExcelFile('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\summary_all_data.xlsx')

xl.sheet_names  # see all sheet names

['percent GDP', 'GDP']

In [119]:
GDP = xl.parse("GDP")  # read a specific sheet to DataFrame

In [120]:
GDP2 = GDP.set_index(["wbregionname","year","ssp"]).gdp.unstack("wbregionname")

In [121]:
GDP2.columns

Index(['East Asia and Pacific', 'Europe and Central Asia',
       'Latin America and Caribbean', 'Middle East and North Africa',
       'South Asia', 'Sub-Saharan Africa'],
      dtype='object', name='wbregionname')

In [122]:
GDP2 = GDP2.rename(columns={"Europe and Central Asia":"Eastern Europe and Central Asia",
                            "Middle East and North Africa":"Middle-East and North Africa"})

In [123]:
GDP2["Africa"] = GDP2["Middle-East and North Africa"]+GDP2["Sub-Saharan Africa"]
GDP2["Asia"] = GDP2["East Asia and Pacific"]+GDP2["South Asia"]
GDP2["all_countries"] = GDP2["Africa"]+GDP2["Asia"]+GDP2["Latin America and Caribbean"]+GDP2["Eastern Europe and Central Asia"]

In [124]:
GDP2.stack().sample(5)

year  ssp   wbregionname                   
2016  ssp2  Eastern Europe and Central Asia    3.362070e+12
2015  ssp1  Sub-Saharan Africa                 2.406483e+12
2023  ssp2  Asia                               2.809969e+13
2019  ssp4  Eastern Europe and Central Asia    3.796818e+12
2018  ssp2  Latin America and Caribbean        8.966850e+12
dtype: float64

In [125]:
res = xl.parse("percent GDP")

In [126]:
res["sector"] = res.sector.replace({"WASH":"wash"})

In [127]:
res.region.unique()

array(['all_countries', 'Africa', 'Asia', 'Latin America and Caribbean',
       'Former soviet Union', 'Middle-East and North Africa',
       'Sub-Saharan Africa', 'South Asia', 'East Asia and Pacific',
       'Eastern Europe and Central Asia'], dtype=object)

In [128]:
res['wbregion'] = res.region.copy()
res.loc[res.region=='Africa','wbregion'] = 'Sub-Saharan Africa'
res.loc[res.region=='Asia','wbregion'] = 'East Asia and Pacific'

res.loc[res.region=='Former soviet Union','wbregion'] = 'Eastern Europe and Central Asia'

disag_afr = res.loc[res.region=='Africa'].copy()
disag_afr.loc[disag_afr.region=='Africa','wbregion'] = 'Middle-East and North Africa'

disag_asia = res.loc[res.region=='Asia'].copy()
disag_asia.loc[disag_asia.region=='Asia','wbregion'] = 'South Asia'

res = res.append(disag_afr).append(disag_asia)

In [129]:
res.loc[res.sector=="energy","ssp"] = "ssp2"
res.loc[(res.sector=="transport")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="transport")&(res.scenario=="min"),"ssp"] = "ssp1"
res.loc[(res.sector=="transport")&(res.scenario=="max"),"ssp"] = "ssp3"
res.loc[(res.sector=="wash")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="wash")&(res.scenario=="min"),"ssp"] = "ssp2"
res.loc[(res.sector=="wash")&(res.scenario=="max"),"ssp"] = "ssp4"
res.loc[(res.sector=="coastal protection")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="coastal protection")&(res.scenario=="min"),"ssp"] = "ssp2"
res.loc[(res.sector=="coastal protection")&(res.scenario=="max"),"ssp"] = "ssp5"
res.loc[(res.sector=="river protection")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="river protection")&(res.scenario=="min"),"ssp"] = "ssp3"
res.loc[(res.sector=="river protection")&(res.scenario=="max"),"ssp"] = "ssp2"
res.loc[(res.sector=="irrigation")&(res.scenario=="best"),"ssp"] = "ssp2"
res.loc[(res.sector=="irrigation")&(res.scenario=="min"),"ssp"] = "ssp1"
res.loc[(res.sector=="irrigation")&(res.scenario=="max"),"ssp"] = "ssp2"

res.loc[(res.region=="all_countries"),"ssp"] = "ssp2"

In [130]:
res[res.ssp.isnull()]

,region,sector,scenario,variable,capital vs maintenance,data,wbregion,ssp


In [131]:
mydata = res.merge(GDP2.stack().reset_index().rename(columns={"wbregionname":"wbregion",0:"GDP"}),on=["wbregion","ssp"])

In [132]:
mydata["cost_bUSD"] = 1e-9/100*mydata.data*mydata.GDP

In [133]:
d = 0.06
mydata["discount_factor"] = (1+d)**(1-(mydata.year.astype(int)-2015))

In [134]:
mydata["cost_bUSD_disc"] = mydata["cost_bUSD"]*mydata["discount_factor"]

In [135]:
newdata = mydata.set_index(["wbregion","sector","scenario","capital vs maintenance","year"]).cost_bUSD_disc.sum(level=["wbregion","sector","scenario","capital vs maintenance"])

In [136]:
newdata2 = newdata.reset_index()

In [137]:
newdata2["cost_bUSD_disc_annual"] = newdata2["cost_bUSD_disc"]/15

In [138]:
final_data = newdata2.merge(res,on=['wbregion','sector','scenario','capital vs maintenance']).drop('cost_bUSD_disc',axis=1).rename(columns={'data':'percent_GDP'})

In [139]:
final_data = final_data[final_data.wbregion!="all_countries"].drop(["variable","ssp","region"],axis=1)

In [140]:
final_data2 = final_data.set_index(['wbregion','sector','scenario','capital vs maintenance']).stack()

In [141]:
final_data2 = final_data2.reset_index().rename(columns={"level_4":"usd vs percent",0:"data"})

In [142]:
final_data3 = final_data2.set_index(['wbregion','sector','scenario','capital vs maintenance',"usd vs percent"]).data.unstack("sector")

In [143]:
final_data3.columns

Index(['coastal protection', 'energy', 'irrigation', 'river protection',
       'transport', 'wash'],
      dtype='object', name='sector')

In [144]:
final_data3['flood protection'] = final_data3['coastal protection']+final_data3['river protection']

final_data3['Total'] = final_data3[['flood protection','energy','transport','irrigation','wash']].sum(axis=1)

In [145]:
final_data4 = final_data3.drop(['coastal protection','river protection'],axis=1).stack().to_frame().reset_index().rename(columns={0:"data"})

In [146]:
sector_dic = {'energy':'Energy','irrigation':'Irrigation','wash':"Water supply and sanitation","transport":"Transport","flood protection":"Flood protection"}

In [147]:
var_dic = {"percent_GDP":"Percent of GDP","cost_bUSD_disc_annual":"Billion USD"}

In [148]:
scenario_dic = {'best':'Preferred','min':'Low spending','max':'High spending'}

In [149]:
final_data4['sector'] = final_data4.sector.replace(sector_dic)
final_data4['usd vs percent'] = final_data4['usd vs percent'].replace(var_dic)
final_data4['scenario'] = final_data4.scenario.replace(scenario_dic)
final_data4['capital vs maintenance'] = final_data4['capital vs maintenance'].str.capitalize()

In [150]:
final_data4.sample(5)

,wbregion,scenario,capital vs maintenance,usd vs percent,sector,data
298,Sub-Saharan Africa,High spending,Maintenance,Billion USD,Energy,6.419665
277,Sub-Saharan Africa,Preferred,Capital,Percent of GDP,Transport,3.200000
313,Sub-Saharan Africa,Low spending,Capital,Percent of GDP,Transport,1.270000
203,Middle-East and North Africa,Low spending,Capital,Percent of GDP,Energy,0.740000
290,Sub-Saharan Africa,High spending,Capital,Billion USD,Transport,159.690540


In [151]:
final_data4.to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\final_for_website.csv')

In [152]:
final_data.sort_values(["wbregion","sector","scenario"]).to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\semi_final_for_website.csv')

In [22]:
totals = pd.DataFrame()

# Energy

In [23]:
energy = newdata2.loc[(newdata2.sector=="energy"),:]

## keep for overview

In [24]:
energy.loc[energy.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()

,scenario,capital vs maintenance,cost_bUSD_disc_annual
0,best,capital,778.114139
1,best,maintenance,206.098170
2,min,capital,298.587895
3,max,capital,1020.534268


In [25]:
temp = energy.loc[energy.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "energy"

In [26]:
totals = temp

In [27]:
energy.loc[energy.region=="all_countries",["scenario","cost_bUSD_disc_annual","capital vs maintenance"]]

,scenario,cost_bUSD_disc_annual,capital vs maintenance
0,min,105.895694,maintenance
5,best,211.791389,maintenance
10,max,282.388519,maintenance
15,min,317.687083,capital
20,best,776.568426,capital
25,max,1058.956944,capital


# Transport

In [28]:
transport = newdata2.loc[(newdata2.sector=="transport"),:]

## keep for overview

In [29]:
transport.loc[transport.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                    416.599217
          maintenance                460.998755
min       capital                    156.796355
          maintenance                552.470203
max       capital                   1060.207794
          maintenance                699.317720
Name: cost_bUSD_disc_annual, dtype: float64

In [30]:
temp = transport.loc[transport.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "transport"

In [31]:
totals = totals.append(temp)

In [32]:
transport.loc[transport.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
1,min,388.284213
6,best,458.881343
11,max,741.269861
16,min,187.082394
21,best,458.881343
26,max,1164.852639


# WASH

In [33]:
wash = newdata2.loc[(newdata2.sector=="wash"),:]

## keep for overview

In [34]:
wash.loc[wash.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                   198.148016
          maintenance                67.185574
min       capital                   115.630826
          maintenance                31.883513
max       capital                   228.925546
          maintenance                68.875119
Name: cost_bUSD_disc_annual, dtype: float64

In [35]:
temp = wash.loc[wash.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "wash"

In [36]:
totals = totals.append(temp)

In [37]:
wash.loc[wash.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
2,min,169.433111
7,best,264.739236
12,max,264.739236
17,min,112.955407
22,best,194.142106
27,max,229.440671


# Flood

In [38]:
coastal_flood = newdata2.loc[(newdata2.sector=="coastal protection"),:]

In [39]:
coastal_flood.loc[coastal_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                   42.919258
          maintenance               21.333604
min       capital                    2.349903
          maintenance                6.152358
max       capital                   56.573056
          maintenance               23.854950
Name: cost_bUSD_disc_annual, dtype: float64

In [40]:
river_flood = newdata2.loc[(newdata2.sector=="river protection"),:]

In [41]:
river_flood.loc[river_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                    60.137460
          maintenance                 3.204862
max       capital                   278.575892
          maintenance                14.260740
min       capital                    20.589857
          maintenance                 1.073654
Name: cost_bUSD_disc_annual, dtype: float64

In [42]:
river_flood.loc[river_flood.region=="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
maxmax    capital                   847.165556
minmin    capital                    14.119426
Name: cost_bUSD_disc_annual, dtype: float64

In [43]:
flood = coastal_flood.loc[coastal_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])+\
river_flood.loc[river_flood.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

## keep for overview

In [44]:
flood

scenario  capital vs maintenance
best      capital                   103.056717
          maintenance                24.538467
max       capital                   335.148948
          maintenance                38.115690
min       capital                    22.939760
          maintenance                 7.226012
Name: cost_bUSD_disc_annual, dtype: float64

In [45]:
temp = flood.reset_index()
temp["sector"] = "flood"

In [46]:
totals = totals.append(temp)

In [47]:
newdata2.loc[(newdata2.sector=="flood"),:]

,region,sector,scenario,capital vs maintenance,cost_bUSD_disc,cost_bUSD_disc_annual
3,all_countries,flood,min,maintenance,74.126986,4.941799
8,all_countries,flood,best,maintenance,370.634931,24.708995
13,all_countries,flood,max,maintenance,741.269861,49.417991
18,all_countries,flood,min,capital,317.687083,21.179139
23,all_countries,flood,best,capital,1694.331111,112.955407
28,all_countries,flood,max,capital,5294.784722,352.985648


# Irrigation

In [48]:
irrigation = newdata2.loc[(newdata2.sector=="irrigation"),:]

## keep for overview

In [49]:
irrigation.loc[irrigation.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                    49.619364
max       capital                   100.434281
min       capital                    43.592279
Name: cost_bUSD_disc_annual, dtype: float64

In [50]:
temp = irrigation.loc[irrigation.region!="all_countries",:].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"]).reset_index()
temp["sector"] = "irrigation"

In [51]:
totals = totals.append(temp)

In [52]:
irrigation.loc[irrigation.region=="all_countries",["scenario","cost_bUSD_disc_annual"]]

,scenario,cost_bUSD_disc_annual
4,min,0.000000
9,best,0.000000
14,max,0.000000
19,min,42.358278
24,best,45.888134
29,max,70.597130


In [53]:
newdata2[(newdata2.region!="all_countries")].set_index(["scenario","capital vs maintenance"]).cost_bUSD_disc_annual.sum(level=["scenario","capital vs maintenance"])

scenario  capital vs maintenance
best      capital                   1545.537453
          maintenance                758.820966
min       capital                    637.547114
max       capital                   2745.250838
min       maintenance                591.579729
max       maintenance                806.308529
Name: cost_bUSD_disc_annual, dtype: float64

In [54]:
newdata2[(newdata2.region=="all_countries")&(newdata2["capital vs maintenance"]=="capital")].set_index("scenario").cost_bUSD_disc_annual.sum(level="scenario")

scenario
min        681.262301
best      1588.435417
max       2876.833032
maxmax     847.165556
minmin      14.119426
Name: cost_bUSD_disc_annual, dtype: float64

## we don't have energy maintenance per region so we use global %

In [55]:
temp2 = energy.loc[(energy.region=="all_countries")&(energy["capital vs maintenance"]=="maintenance")&(energy.scenario!="best"),["scenario","cost_bUSD_disc_annual","capital vs maintenance"]]
temp2["sector"] = "energy"

In [56]:
temp2

,scenario,cost_bUSD_disc_annual,capital vs maintenance,sector
0,min,105.895694,maintenance,energy
10,max,282.388519,maintenance,energy


In [57]:
totals = totals.append(temp2)

C:\ProgramData\Anaconda3\envs\Julie1\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [58]:
totals[totals.sector=="energy"]

,capital vs maintenance,cost_bUSD_disc_annual,scenario,sector
0,capital,778.114139,best,energy
1,maintenance,206.098170,best,energy
2,capital,298.587895,min,energy
3,capital,1020.534268,max,energy
0,maintenance,105.895694,min,energy
10,maintenance,282.388519,max,energy


In [59]:
totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance")

capital vs maintenance      capital  maintenance
sector     scenario                             
energy     best          778.114139   206.098170
           max          1020.534268   282.388519
           min           298.587895   105.895694
flood      best          103.056717    24.538467
           max           335.148948    38.115690
           min            22.939760     7.226012
irrigation best           49.619364          NaN
           max           100.434281          NaN
           min            43.592279          NaN
transport  best          416.599217   460.998755
           max          1060.207794   699.317720
           min           156.796355   552.470203
wash       best          198.148016    67.185574
           max           228.925546    68.875119
           min           115.630826    31.883513

In [60]:
totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").sum(level="scenario")

capital vs maintenance,capital,maintenance
scenario,,
best,1545.537453,758.820966
max,2745.250838,1088.697048
min,637.547114,697.475423


totals.set_index(["sector","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\all_usd_results.csv')

In [61]:
res[res.region=="all_countries"].set_index(["sector","scenario","capital vs maintenance"]).data.unstack("capital vs maintenance")

capital vs maintenance     capital  maintenance
sector           scenario                      
energy           best         2.20        0.600
                 max          3.00        0.800
                 min          0.90        0.300
flood            best         0.32        0.070
                 max          1.00        0.140
                 min          0.06        0.014
irrigation       best         0.13          NaN
                 max          0.20          NaN
                 min          0.12          NaN
river protection maxmax       2.40          NaN
                 minmin       0.04          NaN
transport        best         1.30        1.300
                 max          3.30        2.100
                 min          0.53        1.100
wash             best         0.55        0.750
                 max          0.65        0.750
                 min          0.32        0.480

res[res.region=="all_countries"].set_index(["sector","scenario","capital vs maintenance"]).data.unstack("capital vs maintenance").to_csv('C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\all_percent_results.csv')

In [62]:
newdata2[newdata2.sector=="irrigation"].sort_values("region")

,region,sector,scenario,capital vs maintenance,cost_bUSD_disc,cost_bUSD_disc_annual
177,East Asia and Pacific,irrigation,min,capital,264.913018,17.660868
153,East Asia and Pacific,irrigation,max,capital,529.520308,35.301354
152,East Asia and Pacific,irrigation,best,capital,299.294087,19.952939
75,Eastern Europe and Central Asia,irrigation,max,capital,108.834773,7.255652
74,Eastern Europe and Central Asia,irrigation,best,capital,18.927787,1.261852
93,Eastern Europe and Central Asia,irrigation,min,capital,14.404125,0.960275
54,Latin America and Caribbean,irrigation,best,capital,80.062500,5.337500
55,Latin America and Caribbean,irrigation,max,capital,251.625000,16.775000
90,Latin America and Caribbean,irrigation,min,capital,57.919871,3.861325
174,Middle-East and North Africa,irrigation,min,capital,27.210859,1.814057


In [65]:
region_totals = newdata2.set_index(["region","sector","scenario","capital vs maintenance"]).drop("all_countries",level="region").sum(level=["region","scenario","capital vs maintenance"]).cost_bUSD_disc_annual.unstack("capital vs maintenance").capital

In [70]:
region_totals = region_totals.reset_index()

In [76]:
region_totals['region2'] = region_totals.region

region_totals.loc[region_totals.region=='Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Middle-East and North Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Sub-Saharan Africa','region2']='Africa'
region_totals.loc[region_totals.region=='Asia','region2']='Asia'
region_totals.loc[region_totals.region=='South Asia','region2']='Asia'
region_totals.loc[region_totals.region=='East Asia and Pacific','region2']='Asia'

In [79]:
region_totals2 = region_totals.set_index(['region2','scenario']).capital.sum(level=["region2",'scenario'])

In [82]:
region_totals2 = region_totals2.unstack("scenario")

In [83]:
region_totals2.sum(axis=0)

scenario
best    1545.537453
max     2745.250838
min      637.547114
dtype: float64

In [85]:
region_totalspc = region_totals2/region_totals2.sum(axis=0)

In [87]:
region_totalspc

scenario,best,max,min
region2,,,
Africa,0.204224,0.225473,0.228065
Asia,0.512672,0.504093,0.493327
Eastern Europe and Central Asia,0.118324,0.059585,0.054547
Latin America and Caribbean,0.164781,0.210849,0.224061


faux: ne pas faire le truc ci-dessous (a corriger)

In [136]:
region_pc_totals = mydata.copy()
region_pc_totals['region2'] = region_pc_totals.region

region_pc_totals.loc[region_pc_totals.region=='Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Middle-East and North Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Sub-Saharan Africa','region2']='Africa'
region_pc_totals.loc[region_pc_totals.region=='Asia','region2']='Asia'
region_pc_totals.loc[region_pc_totals.region=='South Asia','region2']='Asia'
region_pc_totals.loc[region_pc_totals.region=='East Asia and Pacific','region2']='Asia'

In [137]:
region_pc_totals.columns

Index(['region', 'sector', 'scenario', 'variable', 'capital vs maintenance',
       'data', 'ssp', 'year', 'GDP', 'cost_bUSD', 'discount_factor',
       'cost_bUSD_disc', 'region2'],
      dtype='object')

In [138]:
region_pc_totals.sample(5)

,region,sector,scenario,variable,capital vs maintenance,data,ssp,year,GDP,cost_bUSD,discount_factor,cost_bUSD_disc,region2
3133,South Asia,river protection,min,% GDP,maintenance,0.0096,ssp3,2028,7.441400e+12,0.714374,0.496969,0.355022,Asia
964,Latin America and Caribbean,river protection,best,% GDP,capital,0.1100,ssp2,2019,9.294505e+12,10.223955,0.839619,8.584230,Latin America and Caribbean
2849,Middle-East and North Africa,coastal protection,max,% GDP,capital,0.0830,ssp5,2016,2.387363e+12,1.981511,1.000000,1.981511,Africa
822,Latin America and Caribbean,wash,best,% GDP,maintenance,0.1600,ssp2,2021,9.948135e+12,15.917016,0.747258,11.894120,Latin America and Caribbean
1971,Sub-Saharan Africa,wash,min,% GDP,maintenance,0.2000,ssp2,2018,2.867385e+12,5.734769,0.889996,5.103924,Africa


In [139]:
region_pc_totals = region_pc_totals.set_index(['sector','scenario','capital vs maintenance','region2','year'])[['cost_bUSD','cost_bUSD_disc','GDP']].sum(level=['scenario','sector','capital vs maintenance','region2','year']).drop("all_countries",level="region2")

In [140]:
region_pc_totals.sample(10)

cost_bUSD  \
scenario sector             capital vs maintenance region2                         year              
max      river protection   maintenance            Asia                            2030   5.745447   
                                                                                   2022   3.811146   
best     energy             maintenance            Eastern Europe and Central Asia 2021  45.142729   
         river protection   capital                Latin America and Caribbean     2016   9.142694   
min      transport          maintenance            Africa                          2018  46.097413   
max      coastal protection capital                Asia                            2030  79.561358   
         wash               maintenance            Eastern Europe and Central Asia 2018   4.746466   
min      transport          maintenance            Eastern Europe and Central Asia 2019  75.612503   
best     river protection   maintenance            Latin America and Caribbean     2028   0.743927   
min      coastal protection capital                Africa                          2019   0.364194   

                                                                                         cost_bUSD_disc  \
scenario sector             capital vs maintenance region2                         year                   
max      river protection   maintenance            Asia                            2030        2.541217   
                                                                                   2022        2.686707   
best     energy             maintenance            Eastern Europe and Central Asia 2021       33.733273   
         river protection   capital                Latin America and Caribbean     2016        9.142694   
min      transport          maintenance            Africa                          2018       41.026533   
max      coastal protection capital                Asia                            2030       35.190065   
         wash               maintenance            Eastern Europe and Central Asia 2018        4.224338   
min      transport          maintenance            Eastern Europe and Central Asia 2019       63.485716   
best     river protection   maintenance            Latin America and Caribbean     2028        0.369709   
min      coastal protection capital                Africa                          2019        0.305784   

                                                                                                  GDP  
scenario sector             capital vs maintenance region2                         year                
max      river protection   maintenance            Asia                            2030  3.988332e+13  
                                                                                   2022  2.670738e+13  
best     energy             maintenance            Eastern Europe and Central Asia 2021  4.103884e+12  
         river protection   capital                Latin America and Caribbean     2016  8.311540e+12  
min      transport          maintenance            Africa                          2018  5.423225e+12  
max      coastal protection capital                Asia                            2030  4.906199e+13  
         wash               maintenance            Eastern Europe and Central Asia 2018  3.651128e+12  
min      transport          maintenance            Eastern Europe and Central Asia 2019  3.799623e+12  
best     river protection   maintenance            Latin America and Caribbean     2028  1.239878e+13  
min      coastal protection capital                Africa                          2019  5.685305e+12

In [141]:
region_pc_totals2 = region_pc_totals[['cost_bUSD','cost_bUSD_disc']].sum(level=['scenario','capital vs maintenance','region2','year'])

In [142]:
region_pc_totals2['GDP'] = region_pc_totals.GDP.mean(level=['scenario','capital vs maintenance','region2','year'])

In [143]:
region_pc_totals2.sample(3)

,,,,cost_bUSD,cost_bUSD_disc,GDP
scenario,capital vs maintenance,region2,year,,,
max,maintenance,Africa,2022,132.774687,93.600915,6.598320e+12
min,maintenance,Asia,2022,567.396689,399.992276,2.678501e+13
best,capital,Africa,2019,363.000115,304.781896,5.685305e+12


In [144]:
region_pc_totals2['new_data'] = 100*1e9*region_pc_totals2.cost_bUSD/region_pc_totals2.GDP

In [145]:
region_pc_totals_mean = region_pc_totals2['new_data'].mean(level=['scenario','capital vs maintenance','region2']).to_frame()

In [150]:
region_pc_totals_mean['cost_bUSD_disc'] = 1/15*region_pc_totals2['cost_bUSD_disc'].sum(level=['scenario','capital vs maintenance','region2'])

In [151]:
region_pc_totals_mean.unstack("capital vs maintenance")

new_data              \
capital vs maintenance                      capital maintenance   
scenario region2                                                  
best     Africa                            6.404040    1.758783   
         Asia                              4.044901    2.538012   
         Eastern Europe and Central Asia   5.797000    3.042000   
         Latin America and Caribbean       3.340000    1.036000   
max      Africa                           12.517166    1.997028   
         Asia                              7.004895    2.688579   
         Eastern Europe and Central Asia   5.166284    2.660183   
         Latin America and Caribbean       7.568284    1.169950   
min      Africa                            2.939233    1.057547   
         Asia                              1.589688    2.144280   
         Eastern Europe and Central Asia   1.098990    2.114121   
         Latin America and Caribbean       1.869217    0.669840   

                                         cost_bUSD_disc              
capital vs maintenance                          capital maintenance  
scenario region2                                                     
best     Africa                              315.635356   86.687476  
         Asia                                792.353132  497.174612  
         Eastern Europe and Central Asia     182.873965   95.963878  
         Latin America and Caribbean         254.675000   78.995000  
max      Africa                              618.981138   98.931500  
         Asia                               1383.861351  533.344619  
         Eastern Europe and Central Asia     163.575944   84.427493  
         Latin America and Caribbean         578.832406   89.604916  
min      Africa                              145.402183   52.155167  
         Asia                                314.519356  421.735824  
         Eastern Europe and Central Asia      34.776291   66.631666  
         Latin America and Caribbean         142.849285   51.057072

In [152]:
region_pc_totals_mean.unstack("capital vs maintenance").to_excel("C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\aggregated_regions_numbers.xlsx")

In [158]:
region_pc_totals3 = 1/15*region_pc_totals[['cost_bUSD_disc']].sum(level=['scenario','capital vs maintenance','region2','sector'])

In [159]:
region_pc_totals3 = region_pc_totals3.reset_index()

In [161]:
region_pc_totals3[(region_pc_totals3.sector=="energy")&(region_pc_totals3["capital vs maintenance"]=="maintenance")]

,scenario,capital vs maintenance,region2,sector,cost_bUSD_disc
1,best,maintenance,Africa,energy,12.817206
7,best,maintenance,Asia,energy,141.042523
13,best,maintenance,Latin America and Caribbean,energy,17.537500
33,best,maintenance,Eastern Europe and Central Asia,energy,34.700942
